In [3]:
# Please do not change this cell because some hidden tests might depend on it.
import os

# Otter grader does not handle ! commands well, so we define and use our
# own function to execute shell commands.
def shell(commands, warn=True):
    """Executes the string `commands` as a sequence of shell commands.
     
       Prints the result to stdout and returns the exit status. 
       Provides a printed warning on non-zero exit status unless `warn` 
       flag is unset.
    """
    file = os.popen(commands)
    print (file.read().rstrip('\n'))
    exit_status = file.close()
    if warn and exit_status != None:
        print(f"Completed with errors. Exit status: {exit_status}\n")
    return exit_status

shell("""
ls requirements.txt >/dev/null 2>&1
if [ ! $? = 0 ]; then
 rm -rf .tmp
 git clone https://github.com/cs187-2021/lab0-1.git .tmp
 mv .tmp/tests ./
 mv .tmp/requirements.txt ./
 rm -rf .tmp
fi
pip install -q -r requirements.txt
""")

In [4]:
# Initialize Otter
import otter
grader = otter.Notebook()

# CS187

## Lab 0-1 – Tensors and vectorization

Many of the data-heavy approaches to NLP are enabled by advances in parallel processing that make what were once intractable computations practical. This notebook demonstrates the issue and the `torch` technologies that apply to it, especially the "tensor" data type.

New bits of Python used for the first time in this lab, and which you may therefore want to review, include:

* `assert`
* `globals`
* `len`
* `math.inf`
* `random.choices`
* `timeit.timeit`
* `torch.tensor`
* `torch.equal`
* `torch.rand`
* `torch.shape`
* `torch.transpose`
* `torch.view`
* `torch.zeros`
* `torch.zeros_like`

In [5]:
import torch
import random

from timeit import timeit

Numeric vectors can be implemented in Python in many ways. Most directly, Python provides a built-in `list` data type, which we could use to implement a vector. Here, we generate a couple of example vectors as lists each containing 1000 integers between 0 and 99.

In [6]:
a1 = random.choices(range(100), k=1000)
a2 = random.choices(range(100), k=1000)

# An example: dot product

The dot product of two vectors $v$ and $w$ is the sum of their componentwise product, $\sum_i v_i \cdot w_i$, which can be calculated with a simple for-loop.

In [7]:
def dotproduct(v, w):
    assert len(v) == len(w)
    sum = 0
    for i in range(len(v)):
         sum += v[i] * w[i]
    return sum

In [8]:
dotproduct_result = dotproduct(a1, a2)
dotproduct_result

2481804

We can test the efficiency of this approach to implementing vectors by computing a large dot product many times. (We use the `timeit` function that we imported from the `timeit` library above to return the time in seconds to perform 100,000 repetitions of the dot product computation.)

In [9]:
example_time = timeit('dotproduct(a1, a2)', number=100000, globals=globals())
print(f"It took {example_time:.3} seconds.")

It took 16.3 seconds.


As it turns out, performing this vector computation is quite slow -- it probably took several seconds -- because the `for` loop over the list data structure computes sequentially. Instead, we can use a data type engineered especially for such vector and array computations to improve performance. Such data types include Python arrays, `numpy` arrays, and [`torch` tensors](https://pytorch.org/docs/stable/tensors.html). The latter are especially designed for the kinds of computations found in machine learning algorithms, so we will use them throughout the course. You can read (a lot) more about tensors in [the official documentation](https://pytorch.org/docs/stable/tensors.html).

We construct a couple of one-dimensional tensors for the examples above.

In [10]:
t1 = torch.tensor(a1)
t2 = torch.tensor(a2)

# Tensor properties

Tensors have four characteristics that are especially useful (but potentially confusing when you first start working with them):

+ Componentwise operation: Many operations on tensors work component by component instead of all at once.
+ Broadcast: Operations can broadcast individual elements to each component of a tensor.
+ Reshaping: Tensors can be reshaped to present the same elements in different configurations.
+ Special operations: Tensors have methods implementing certain operations especially efficiently.

We give examples of each.

## Componentwise operation

Many operations on tensors work [*componentwise*](https://en.wikipedia.org/wiki/Pointwise#Componentwise_operations), that is, separately for each component of the tensor, rather than on the tensor all at once.

For instance, when we add two tensors of the same shape with the `+` operator, the summation percolates down to the individual comonents. For example,

In [11]:
a3 = [1, 2, 3]
a4 = [4, 5, 6]
t3 = torch.tensor(a3)
t4 = torch.tensor(a4)

t3 + t4

tensor([5, 7, 9])

This is quite different from, say, lists, which perform a completely different operation – concatenation – when summed with the `+` operator.

In [12]:
a3 + a4

[1, 2, 3, 4, 5, 6]

Similarly, we can compute the elementwise product of two tensors of the same shape.

In [13]:
t3 * t4

tensor([ 4, 10, 18])

## Broadcast

Related, adding a scalar to a tensor "broadcasts" the scalar addition operation to each element.

In [14]:
print(t3 + 5)
print(5 + t3)

tensor([6, 7, 8])
tensor([6, 7, 8])


Again, compare with how lists work (or actually, don't work). Try uncommenting and running the cell below.

In [15]:
a3 + 5

TypeError: can only concatenate list (not "int") to list

## Reshaping

Finally, tensors can be reshaped so that their elements appear in a different configuration. The `view` method is often used to carry out the reshaping. For instance, we start with the following 3 by 4 tensor.

In [16]:
t5 = torch.tensor([[11, 12, 13, 14],
                   [21, 22, 23, 24],
                   [31, 32, 33, 34]])

We can tell that `t5` is a 3 by 4 tensor using the `shape` method.

In [17]:
t5.shape

torch.Size([3, 4])

We can view the elements as a 4 by 3 tensor, or a 2 by 2 by 3 tensor, or a 3 by 1 by 4 tensor.

In [18]:
print(t5.view(4, 3))
print(t5.view(2, 2, 3))
print(t5.view(3, 1, 4))

tensor([[11, 12, 13],
        [14, 21, 22],
        [23, 24, 31],
        [32, 33, 34]])
tensor([[[11, 12, 13],
         [14, 21, 22]],

        [[23, 24, 31],
         [32, 33, 34]]])
tensor([[[11, 12, 13, 14]],

        [[21, 22, 23, 24]],

        [[31, 32, 33, 34]]])


## Special operations

Tensors have a large set of methods defined on them that work especially efficiently, for instance, taking the sum of the elements, or the minimum.

In [19]:
t5.sum()

tensor(270)

In [20]:
t5.min()

tensor(11)

The `min` method takes the minimum over all the elements in the tensor. But we often want to take the minimum or maximum with respect to a particular dimension, returning a tensor of these optima. For example, given a two-dimensional tensor, to find the minimum for each row, we can take the `min` with respect to the second dimension. (We refer to the row dimension as `1` since dimensions are 0-indexed.) 

For some intuition (especially helpful as the number of dimensions gets higher and you lose the simple notion of "rows" and "columns"), think of the `.min` by dimension operation as collapsing that dimension - for example, if you take the `.min(1)` of a $4 \times 5 \times 6$ tensor, your output will be a $4 \times 6$ tensor. 

The `min` method called in this way returns  a tensor of the various minimum values, along with the indices at which these minima occurred. We can extract the `values` if we are interested only in those. (Feel free to take a look at the `indices` part of the return value to see what that looks like. Does its value make sense?)

In [21]:
t5.min(1).values

tensor([11, 21, 31])

<!--
BEGIN QUESTION
name: max_val
-->
Define a function to find the maximum values for each column of a two-dimensional tensor.

In [22]:
# TODO -- Implement a function to return the max value of each column, stored in a 1-D tensor.
def max_col(v):
    col_max = v.max(0).values
    return col_max

In [23]:
grader.check("max_val")

All tests passed!

# Vectorized dot product

<!--
BEGIN QUESTION
name: dotprod
-->
Using these tensor techniques, and noting the examples above, reimplement a version of `dotproduct` that has no `for` loops. **Hint:** Your code should be *very short*.

In [24]:
# TODO -- Implement a vectorized dot product, which should be much faster.
def dotproduct_v(v1, v2):
    dot_product = torch.sum(v1 * v2)
    return dot_product 

In [25]:
grader.check("dotprod")

All tests passed!

This vectorized version should be *much* faster, perhaps a couple of orders of magnitude.

In [26]:
example_time_v = timeit('dotproduct_v(t1, t2)', number=100000, globals=globals())
print(f"It took {example_time_v:.3} seconds.")

It took 0.615 seconds.


# Vectorized computations over multidimensional tensors

As you saw above, tensors aren't limited to one dimension, and these same vectorization tricks apply to multidimensional tensors. In fact, because of vectorization, we can specify computations over multidimensional tensors that look like the kinds of things you've seen in linear algebra. 

#### An example: all-paths shortest path

<img src="https://github.com/nlp-course/data/raw/master/Resources/small-graph.png" width=200 align=right />
As a concrete example to give you some practice, consider the algorithm for computing shortest paths in a graph of $n$ nodes. We'll represent the graph as an $n \times n$ matrix $A$ where $A_{ij}$ is the distance from node $i$ to node $j$. (Thus, this is a directed graph, and the distances needn't be symmetric.) We'll assume that the distance from a node to itself is zero. Here's an example, with distances that happen to be symmetric:

In [27]:
from math import inf
distances = torch.tensor(
              [[0, 1,   2, 6],
               [1, 0,   2, inf],
               [2, 2,   0, 3],
               [6, inf, 3, 0]])

(We use `inf` for an infinite distance, that is, for nodes that are not connected with an edge.) 

In this graph, the distance from node 0 to node 3 is 6, but by going through node 2, we can shorten the path to 5. In general, we define [the *minplus* operation](https://en.wikipedia.org/wiki/Min-plus_matrix_multiplication) ($\star$) on two square matrices $A$ (of shape $m \times n$) and $B$ (of shape $n \times p$):
$$(A \star B)_{ij} = \min_k A_{ik} + B_{kj}$$
As a special case, if $A$ and $B$ are two distance graphs over the same nodes (but perhaps with different distances), then $A \star B$ is the graph that shows the best way to get from one node to another by traversing an edge from the first graph $A$ and then an edge from the second graph $B$.

Here's an implementation of this operation `minplus` using `for` loops.

In [28]:
def minplus_loop(A, B):
    (arows, acols), (brows, bcols) = A.size(), B.size()
    assert acols == brows

    R = torch.zeros(arows, bcols)
    for i in range(arows):
        for j in range(bcols):
            min = torch.tensor(inf)
            for k in range(acols):
                if A[i,k] + B[k,j] < min:
                    min = A[i,k] + B[k,j]
            R[i,j] = min
    return R

Let's test it out on a small rectangular matrix.

In [29]:
test = torch.tensor([[1, 2, 3], [4, 5, 6]])
minplus_loop(test, test.transpose(0, 1))

tensor([[2., 5.],
        [5., 8.]])

Using this, we can compute some better ways of getting among the nodes in the`distances` graph. For paths of length at most 2, we can compute

In [30]:
minplus_loop(distances, distances)

tensor([[0., 1., 2., 5.],
        [1., 0., 2., 5.],
        [2., 2., 0., 3.],
        [5., 5., 3., 0.]])

Notice that in this graph, the distance from node 0 to node 3 is now only 5 (not 6), and there are also now paths between nodes 1 and 3.

We can compute the minimum distance between any two nodes by repeating this minplus process until no further distance reductions are possible and the graph has reached a stable point (the so-called "fixpoint"). We return as a Python tuple both the fixpoint graph and the number of rounds of `minplus` that were needed to reach it.

In [31]:
def minplus_fp(X):
    rounds = 0
    lastY = torch.zeros_like(X)
    Y = X
    while not(torch.equal(Y, lastY)):
        lastY = Y
        Y = minplus_loop(Y, Y)
        rounds += 1
    return Y, rounds

In [32]:
minplus_fp(distances)

(tensor([[0., 1., 2., 5.],
         [1., 0., 2., 5.],
         [2., 2., 0., 3.],
         [5., 5., 3., 0.]]),
 2)

It turns out that after just the two rounds, the fixpoint is reached.

> **Digression:** The complexity of `minplus` as implemented is $O(n^3)$, and the fixpoint computation may need up to $\log n$ calls to `minplus` to converge, so the overall complexity is $O(n^3 \log n)$. More efficient algorithms are known, especially the Floyd-Warshall algorithm for the all-paths versions and Dijkstra's algorithm for the single-source version. But algorithmic efficiency is not our main aim here.

Let's try a bigger example, a graph with 10 nodes.

In [33]:
def random_square_tensor(size):
    X = torch.rand(size, size)
    for i in range(size):
        X[i, i] = 0 
    return X

X = random_square_tensor(10)
X

tensor([[0.0000e+00, 4.7034e-01, 9.2198e-01, 1.1048e-01, 9.0072e-01, 5.8341e-01,
         8.8688e-01, 3.0471e-02, 4.6556e-02, 3.4740e-01],
        [8.0501e-01, 0.0000e+00, 2.5678e-01, 3.8154e-01, 4.9806e-01, 4.1833e-01,
         1.0529e-01, 5.3634e-01, 7.5587e-01, 4.5685e-01],
        [7.4277e-01, 4.2446e-01, 0.0000e+00, 6.2463e-01, 4.2040e-01, 5.8082e-01,
         7.8946e-01, 5.4890e-01, 8.3928e-01, 8.0930e-01],
        [6.1468e-01, 9.5774e-01, 8.6574e-01, 0.0000e+00, 7.8742e-01, 8.2762e-01,
         7.8308e-01, 4.6647e-01, 8.8889e-01, 8.3725e-01],
        [8.4139e-01, 5.2283e-01, 5.7243e-01, 7.6203e-01, 0.0000e+00, 2.3787e-01,
         1.9251e-01, 9.1143e-01, 6.3736e-01, 7.8032e-01],
        [1.0990e-01, 1.5892e-01, 4.5636e-01, 2.0964e-01, 8.8170e-01, 0.0000e+00,
         4.0529e-01, 9.6520e-01, 8.2855e-01, 1.1021e-04],
        [1.2128e-01, 1.1308e-01, 3.1515e-01, 2.3179e-02, 7.4975e-01, 1.1700e-01,
         0.0000e+00, 2.3435e-01, 3.3697e-01, 5.9543e-01],
        [8.1584e-01, 3.7192

In [34]:
minplus_fp(X)

(tensor([[0.0000e+00, 6.7663e-02, 1.0545e-01, 1.1048e-01, 2.3692e-01, 1.7146e-01,
          1.7295e-01, 3.0471e-02, 4.6556e-02, 7.5375e-02],
         [2.2657e-01, 0.0000e+00, 2.5248e-01, 1.2847e-01, 4.6349e-01, 2.2229e-01,
          1.0529e-01, 2.5704e-01, 2.7313e-01, 2.2240e-01],
         [6.5103e-01, 4.2446e-01, 0.0000e+00, 5.5292e-01, 4.2040e-01, 5.8082e-01,
          5.2974e-01, 5.4890e-01, 6.9758e-01, 5.8093e-01],
         [6.1468e-01, 5.0366e-01, 5.4145e-01, 0.0000e+00, 7.8742e-01, 7.2595e-01,
          6.0895e-01, 4.6647e-01, 6.6124e-01, 5.1137e-01],
         [3.1379e-01, 3.0558e-01, 2.6805e-01, 2.1569e-01, 0.0000e+00, 2.3787e-01,
          1.9251e-01, 3.4427e-01, 3.6035e-01, 2.3798e-01],
         [1.0990e-01, 1.5892e-01, 3.0186e-02, 1.6098e-01, 3.4681e-01, 0.0000e+00,
          1.3780e-01, 1.4037e-01, 1.5645e-01, 1.1021e-04],
         [1.2128e-01, 1.1308e-01, 1.4719e-01, 2.3179e-02, 3.5820e-01, 1.1700e-01,
          0.0000e+00, 1.5176e-01, 1.6784e-01, 1.1711e-01],
         [2.6

Now let's talk about the "loop"-y implementation of the `minplus` function. If we're a little cleverer, we can use list comprehensions to hide the computation of the minimum, but we're still doing the whole computation sequentially.

In [35]:
def minplus_loop2(A, B):
    (arows, acols), (brows, bcols) = A.size(), B.size()
    assert acols == brows

    R = torch.zeros(arows, bcols)
    assert acols == brows
    for i in range(arows):
        for j in range(bcols):
            R[i,j] = min([A[i,k] + B[k,j] for k in range(acols)])
    return R

minplus_loop2(test, test.transpose(0,1))

tensor([[2., 5.],
        [5., 8.]])

The `torch`-y way to perform this computation is to rely on vectorized computations. Doing so is a bit tricky however. We need to reshape the matrices a bit. We start by turning the two-dimensional $A$ matrix from a $m \times n$ matrix (rows by columns, which may differ in the general case) into a three-dimensional $m \times 1 \times n$ matrix. Here's the result of that operation on the $4 \times 4$ `distances` matrix, using [the torch `view` method](https://pytorch.org/docs/stable/tensor_view.html). We'll refer to the reshaped matrix below as the $A$ matrix henceforth. It will now be a tensor with 4 elements, each of which has 1 element, each of which has 4 elements (that are scalars).

In [36]:
A = distances.view(4, 1, 4)
A

tensor([[[0., 1., 2., 6.]],

        [[1., 0., 2., inf]],

        [[2., 2., 0., 3.]],

        [[6., inf, 3., 0.]]])

Now each row in the matrix contains a single element, a vector that corresponds to the column in the original. We'll do a similar operation on $B$ of shape $n \times p$ (again, the `distances` matrix in our example), but this time reshaping the matrix to be $1 \times p \times n$. We'll refer to the reshaped matrix below as $B$. It will now be a tensor with just 1 element, and that element contains 4 elements, each of which contains 4 elements.

In [39]:
B = distances.view(1, 4, 4)
B

tensor([[[0., 1., 2., 6.],
         [1., 0., 2., inf],
         [2., 2., 0., 3.],
         [6., inf, 3., 0.]]])

Now if we add these two matrices componentwise, what do we get? Each of the four 1 $\times$ 4 elements in $A$ corresponds to the same single 4 $\times$ 4 element in $B$, so they'll be added componentwise; that single element in $B$ will be "broadcast" to each of the rows in $A$. Thus, the first element in $A$ (`[[0., 1., 2., 6.]]` in the example) is to be added to the single element in $B$ (`[[0., 1., 2., 6.], [1., 0., 2., inf], [2., 2., 0., 3.], [5., inf, 3., 0.]]` in the example). How does this work? Going one more level in, we now have that the single 4-element element `[0., 1., 2., 6.]` in $A$ corresponds to (and will be broadcast to) each of the four 4-element elements in the single element in $B$ (the first of which is also `[0., 1., 2., 6.]`). Now, we've reached a set of elements that are all the same size as the element to be broadcast, so they are added elementwise, yielding `[0., 2., 4., 12.]` as the first summed 4-element element. The same thing happens for each of the other three elements in the reshaped $B$ matrix element. Zooming out one step, we repeat this procedure for each of the other three 1 $\times$ 4 elements of $A$, broadcasting them onto the single 4 $\times$ 4 element of $B$. When all is said and done, we'll have an $m \times p \times n$ matrix. Each outermost element of the summed matrix corresponds to a broadcast of a 1 $\times$ 4 element of $A$ onto the 4 $\times$ 4 element of $B$.

As another way to understand this process, we want to compute the elementwise sum of a matrix $A$ of size 4 $\times$ 1 $\times$ 4 with another matrix $B$ of size 1 $\times$ 4 $\times$ 4. If both matrices were of size 4 $\times$ 4 $\times$ 4, there would be no problem. However, the first dimension of $A$ is of size 4 whereas the first dimension of $B$ is of size 1. Therefore, we repeat $B$ 4 times along the first dimension, expanding it into a matrix of size 4 $\times$ 4 $\times$ 4. Now the first dimension works out, but the second dimension of $A$ is of size 1 whereas the second dimension of $B$ is of size 4, so we also need to repeat $A$ 4 times along the second dimension, expanding it into a matrix of 4 $\times$ 4 $\times$ 4. Now both $A$ and $B$ have been expanded into size 4 $\times$ 4 $\times$ 4, and we can directly compute their elementwise sum. This hidden expanding operation is exactly the broadcasting we've referred to above.

In [40]:
summed = distances.view(4, 1, 4) + distances.view(1, 4, 4)
summed

tensor([[[ 0.,  2.,  4., 12.],
         [ 1.,  1.,  4., inf],
         [ 2.,  3.,  2.,  9.],
         [ 6., inf,  5.,  6.]],

        [[ 1.,  1.,  4., inf],
         [ 2.,  0.,  4., inf],
         [ 3.,  2.,  2., inf],
         [ 7., inf,  5., inf]],

        [[ 2.,  3.,  2.,  9.],
         [ 3.,  2.,  2., inf],
         [ 4.,  4.,  0.,  6.],
         [ 8., inf,  3.,  3.]],

        [[ 6., inf,  5.,  6.],
         [ 7., inf,  5., inf],
         [ 8., inf,  3.,  3.],
         [12., inf,  6.,  0.]]])

How does this matrix translate to the $A_{ik} + B_{kj}$ additions we did before? To illustrate by example (indexing starting from 0), let's think about $(A \star B)_{1, 2} = \min_k A_{1, k} + B_{k, 2}$, where $A$ and $B$ now refer to the original 4 $\times$ 4 `distances` matrix that we started with. The $A_{1, k}$s would be found in the second element of `summed`, which is where we broadcasted the second (index 1) row of $A$. Since this matrix is symmetric, the $B_{k, 2}$'s would be equivalent to the $B_{2, k}$'s, which would be found in the second row of each element of `summed`. Taking these two pieces of information together, we then look at the second row of the second element of `summed`. Indeed, $A_{1, 0} + B_{0, 2} = A_{1, 0} + B_{2, 0}$ would be the first element in the third row of the second element of `summed`, and this is true for each $k$. This tells us that to find our `minplus` result, we just need to find the minimum of each row of every element of `summed` (using the `min` method), and there are $4 \cdot 4$ such rows. Note that this solution only works for symmetric matrices - for non-symmetric matrices, you'll need to transpose `distances` before you reshape it into a $1 \times 4 \times 4$ tensor.

<!--
BEGIN QUESTION
name: minplus_v_example
-->
Calculate the result of the `minplus` by performing appropriate operations on `summed` to yield a tensor that should be identical to the `minplus_loop(distances, distances)` example above.

In [41]:
#TODO find the minimum of each row of every element of summed (using the min method)
result = summed.min(-1).values 
result

tensor([[0., 1., 2., 5.],
        [1., 0., 2., 5.],
        [2., 2., 0., 3.],
        [5., 5., 3., 0.]])

In [42]:
grader.check("minplus_v_example")

All tests passed!

Now that you've seen an example of how the matrices can be reshaped and operated on to implement the `minplus` operation, write a function `minplus_v` (a "vectorized" version of `minplus_loop`) that computes the minplus of two rectangular matrices without using any looping constructs.
<!--
BEGIN QUESTION
name: minplus_v
-->

In [55]:
#TODO
def minplus_v(A, B):
    
    #Ensure match
    (arows, acols), (brows, bcols) = A.size(), B.size()
    assert acols == brows
    
    #Reshaping 𝐴 from a  𝑚×𝑛 to 𝑚×1×𝑛 matrix.
    A = A.view(A.size(0), 1, A.size(1))
    
    #Reshaping B to be  1×𝑝×𝑛
    B = B.view(1, B.size(0), B.size(1))
    
    #Find the minimum of each row of every element
    summed = A + B
    result = summed.min(-1).values 
    
    return result

In [56]:
grader.check("minplus_v")

All tests passed!

Finally, we'll use your vectorized `minplus_v` to implement a vectorized fixpoint calculation, and test the relative speeds.

In [57]:
def minplus_vfp(X):
    rounds = 0
    lastY = torch.zeros_like(X)
    Y = X
    while not(torch.equal(Y, lastY)):
        lastY = Y
        Y = minplus_v(Y, Y)
        rounds += 1
    return Y, rounds

In [58]:
minplus_vfp(distances)

(tensor([[0., 1., 2., 5.],
         [1., 0., 2., 5.],
         [2., 2., 0., 3.],
         [5., 5., 3., 0.]]),
 2)

In [59]:
example = random_square_tensor(20)

timeit('minplus_fp(example)', number=10, globals=globals())

4.15391108099999

In [60]:
timeit('minplus_vfp(example)', number=10, globals=globals())

0.01098165599995582

If you've implemented `minplus_v` correctly, the efficiency difference should be striking. This kind of engineered improvement is the difference between a computation taking a day and one taking a minute.

# Submission Instructions

This lab should be submitted to Gradescope at <http://go.cs187.info/lab0-1-submit>, which will be made available some time before the due date.

Make sure that you have passed all public tests by running `grader.check_all()` below before submitting. Note that there are hidden tests on Gradescope, the results of which will be revealed after the submission deadline.

# End of lab 0-1

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [61]:
grader.check_all()

dotprod:

    All tests passed!
    

max_val:

    All tests passed!
    

minplus_v:

    All tests passed!
    

minplus_v_example:

    All tests passed!